In [1]:
#df = pd.read_csv("sentiment140.csv", encoding = "cp1252", header=None)
#df.columns =['score', 'id', 'time', 'query', 'user_name', 'comment']
#df = df.drop(['id', 'time', 'query', 'user_name'], axis=1)
#df.comment.isnull().sum()

#score_cnt = df.groupby("score")["comment"].count()/len(df)
# The score has only 0 and 4, so change 4 into 1.
#df.loc[df.score == 4, 'score'] = 1
#score_cnt_new = df.groupby("score")["comment"].count()/len(df)

#from nltk import word_tokenize

# Tokenize each item in the review column
#word_tokens = [word_tokenize(com) for com in df.comment]

# Create an empty list to store the length of the reviews
#len_tokens = []

# Iterate over the word_tokens list and determine the length of each item
#for i in range(len(word_tokens)):
     #len_tokens.append(len(word_tokens[i]))

# Create a new feature for the lengh of each review
#df['n_words'] = len_tokens
#df.to_csv("sentiment140_new.csv", encoding="utf-8")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv("/content/drive/MyDrive/Redback_A/sentiment140_new.csv")
data = data.drop("Unnamed: 0", axis=1)
data.head()

,score,comment,n_words
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",27
1,0,is upset that he can't update his Facebook by ...,25
2,0,@Kenichan I dived many times for the ball. Man...,21
3,0,my whole body feels itchy and like its on fire,10
4,0,"@nationwideclass no, it's not behaving at all....",30


In [5]:
# The dataset is very large, select only part of it
from sklearn.model_selection import train_test_split

# Stratify the dataset by the 'target' column and split it into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.7, stratify=data['score'])
df = test_df.copy()

In [6]:
# Import the TfidfVectorizer and default list of English stop words
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk import word_tokenize

# Build the vectorizer
vect = TfidfVectorizer(stop_words=list(ENGLISH_STOP_WORDS), 
                       ngram_range=(1, 2), 
                       max_features=50, 
                       token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(df.comment)
# Create sparse matrix from the vectorizer
X = vect.transform(df.comment)

# Create a DataFrame
reviews_transformed = pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out())

In [7]:
reviews_transformed.head()

,amp,bad,bit,com,come,day,did,don,feel,fun,...,think,time,today,tomorrow,tonight,twitter,ve,want,wish,work
0,0.719471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = df.drop('comment', axis=1)
df.head()

,score,n_words
1537668,1,21
129326,0,32
842500,1,4
1153526,1,14
783903,0,16


In [9]:
reviews_transformed = reviews_transformed.merge(df, left_index=True, right_index=True)

In [10]:
reviews_transformed.columns

Index(['amp', 'bad', 'bit', 'com', 'come', 'day', 'did', 'don', 'feel', 'fun',
       'going', 'good', 'got', 'great', 'haha', 'happy', 'home', 'hope',
       'http', 'im', 'just', 'know', 'like', 'll', 'lol', 'love', 'make',
       'miss', 'morning', 'need', 'new', 'night', 'oh', 'quot', 'really',
       'right', 'sad', 'sleep', 'sorry', 'thanks', 'think', 'time', 'today',
       'tomorrow', 'tonight', 'twitter', 've', 'want', 'wish', 'work', 'score',
       'n_words'],
      dtype='object')

In [11]:
print(len(reviews_transformed.index))
print(reviews_transformed.shape)

784095
(784095, 52)


In [12]:
reviews_transformed.score.unique()

array([0, 1])

In [13]:
from sklearn.model_selection import train_test_split, cross_val_score

# Define X and y
y = reviews_transformed.score
X = reviews_transformed.drop('score', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=456)

In [14]:
# Apply PCA for feature reduction
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [15]:
(X_train_pca < 0).any().any()

True

In [16]:
# Import the Support Vector Machine
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# Create a SVM classifier
#svm = SVC(kernel='linear', C=1.0)
#scores = cross_val_score(svm, X_train_pca, y_train, cv=3)

# Evaluate the performance of the model using cross-validation
#print('Cross-validation scores: {}'.format(scores))
#print('Average cross-validation score: {:.2f}'.format(scores.mean()))

In [17]:
# Use grid_search to find the optimal C
# from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the SVM model
# param_grid = {'C': [0.1, 1, 5]}

# Define the SVM model with a linear kernel
# svm = SVC(kernel='linear')

# Perform grid search cross-validation to find the optimal value of C
# grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=3)
# grid_search.fit(X_train_pca, y_train)

# Print the results
# print("Best C:", grid_search.best_params_['C'])
# print("Best score:", grid_search.best_score_)

In [ ]:
# Fit the classifier to the training data, using the best C value
svm = SVC(kernel='linear', C=0.1)
svm.fit(X_train_pca, y_train)

# Create the predicted tags: pred
pred = svm.predict(X_test_pca)

# Print accuracy score and confusion matrix on test set
print('Accuracy on the test set: ', accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred)/len(y_test))

In [ ]:
# Get the class labels: class_labels
class_labels = svm.classes_

# Extract the features: feature_names
feature_names = vect.get_feature_names_out()

# Zip the feature names together with the coefficient array and sort by weights: feat_with_weights
feat_with_weights = sorted(zip(svm.coef_[0], feature_names))

# Print the first class label and the top 5 feat_with_weights entries
print(class_labels[0], feat_with_weights[:5])

# Print the second class label and the bottom 5 feat_with_weights entries
print(class_labels[1], feat_with_weights[-5:])